# Capítulo 5: Máquinas vectoriales de soporte (SVM)

</br></br>
Una máquina de vectores de soporte (SVM) es un modelo de aprendizaje automático potente y versátil, capaz de realizar la clasificación **lineal o no lineal**, la regresión e incluso la detección de novedades. 

Los SVM **funcionan muy bien con conjuntos de datos no lineales de tamaño pequeño a mediano** (es decir, de cientos a miles de instancias), especialmente para tareas de clasificación. Sin embargo, no se escalan muy bien a conjuntos de datos muy grandes, como verás.

Este capítulo explicará los conceptos básicos de los SVM, cómo usarlos y cómo funcionan. ¡Vamos a entrar!

## Clasificación lineal de SVM

</br></br>
La idea fundamental detrás de los SVM se explica mejor con algunas imágenes. 

La Figura 5-1 muestra parte del conjunto de datos del iris que se introdujo al final del capítulo 4. 
Las dos clases se pueden separar fácilmente con una línea recta (son separables linealmente). 

El gráfico de la izquierda muestra los límites de decisión de tres posibles clasificadores lineales. 
El modelo cuyo límite de decisión está representado por la línea discontinua es tan malo que ni siquiera separa las clases correctamente. 

Los otros dos modelos funcionan perfectamente en este conjunto de entrenamiento, pero sus límites de decisión se acercan tanto a las instancias que estos modelos probablemente no funcionarán tan bien en las nuevas instancias. 

Por el contrario, la línea continua en el gráfico de la derecha representa el límite de decisión de un clasificador SVM; esta línea no solo separa las dos clases, sino que también se mantiene lo más lejos posible de las instancias de entrenamiento más cercanas. 

Se puede pensar en un clasificador SVM que se ajusta a la calle más ancha posible (representada por las líneas discontinuas paralelas) entre las clases. Esto se llama **clasificación de gran margen**.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0501.png)

(_Figura 5-1. Clasificación de gran margen_)

Tenga en cuenta que agregar más instancias de entrenamiento "fuera de la calle" no afectará en absoluto al **límite de la decisión: está totalmente determinado (o "apoyado") por las instancias ubicadas en el borde de la calle**. Estas instancias se llaman **VECTORES DE SOPORTE** (están rodeados en la Figura 5-1).

#### --------------------------- ADVERTENCIA -------------------------------------
Las SVM son sensibles a las escalas de características, como puede ver en la Figura 5-2. En el gráfico de la izquierda, la escala vertical es mucho mayor que la escala horizontal, por lo que la calle más ancha posible está cerca de la horizontal. Después del escalado de características (por ejemplo, usando StandardScaler de Scikit-Learn), el límite de decisión en el gráfico correcto se ve mucho mejor.
#### -----------------------------------------------------------------------------------

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0502.png)

(_Figura 5-2. Sensibilidad a las escalas de características_)

## Clasificación de margen blando (soft-margin)

Si imponemos estrictamente que todas las instancias deben estar fuera de la calle y en el lado correcto, esto se llama clasificación de margen duro. Hay dos problemas principales con la clasificación del margen duro. En primer lugar, solo funciona si los datos son separables linealmente. En segundo lugar, es sensible a los valores atípicos. La figura 5-3 muestra el conjunto de datos del iris con solo un valor atípico adicional: a la izquierda, es imposible encontrar un margen duro; a la derecha, el límite de decisión termina siendo muy diferente al que vimos en la Figura 5-1 sin el valor atípico, y el modelo probablemente no se generalizará también.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0503.png)

(_Figura 5-3. Sensibilidad al margen duro a los valores atípicos_)

Para evitar estos problemas, necesitamos utilizar un modelo más flexible. El objetivo es encontrar un buen equilibrio entre mantener la calle lo más grande posible y limitar las violaciones de los márgenes (es decir, los casos que terminan en el medio de la calle o incluso en el lado equivocado). Esto se llama clasificación de margen blando.

Al crear un modelo SVM usando Scikit-Learn, puede especificar varios hiperparámetros, incluido el hiperparámetro de regularización `C`. Si lo establece en un valor bajo, terminará con el modelo a la izquierda de la Figura 5-4. 
Con un valor alto, obtienes el modelo de la derecha. Como puede ver, reducir `C` hace que la calle sea más grande, pero también genera más violaciones de márgenes. En otras palabras, reducir `C` da como resultado que haya más instancias que apoyen la calle, por lo que hay menos riesgo de sobreajuste. Pero si lo reduce demasiado, entonces el modelo termina por no ajustarse lo suficiente, como parece ser el caso aquí: parece que el modelo con `C=100` se generalizará mejor que el modelo con `C=1`.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0504.png)

(_Figura 5-4. Gran margen (izquierda) frente a menos violaciones de margen (derecha)_)

#### ------------------------------- PROPINA ---------------------------------------
Si su modelo SVM está demasiado ajustado, puede intentar regularizarlo reduciendo C.
#### -----------------------------------------------------------------------------------

El siguiente código Scikit-Learn carga el conjunto de datos de iris y entrena un clasificador SVM lineal para detectar flores de Iris virginica. La canalización primero escala las características y luego usa un `LinearSVC` con `C=1`:

In [1]:
from sklearn.datasets import load_iris
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = load_iris(as_frame=True)
X = iris.data[["petal length (cm)", "petal width (cm)"]].values
y = (iris.target == 2)  # Iris virginica

svm_clf = make_pipeline(StandardScaler(),
                        LinearSVC(C=1, random_state=42))
svm_clf.fit(X, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(C=1, random_state=42))])

El modelo resultante está representado a la izquierda en la Figura 5-4.

Entonces, como de costumbre, puedes usar el modelo para hacer predicciones:

In [2]:
X_new = [[5.5, 1.7], [5.0, 1.5]]
svm_clf.predict(X_new)

array([ True, False])

La primera planta se clasifica como Iris virginica, mientras que la segunda no lo es. Echemos un vistazo a las puntuaciones que el SVM utilizó para hacer estas predicciones. Estos miden la distancia firmada entre cada instancia y el límite de la decisión:

In [3]:
svm_clf.decision_function(X_new)

array([ 0.66163411, -0.22036063])

A diferencia de `LogisticRegression`, `LinearSVC` no tiene un método `predict_proba()` para estimar las probabilidades de clase. Dicho esto, si usa la clase `SVC` (que se analiza en breve) en lugar de `LinearSVC`, y si establece su hiperparámetro de `probability` en `True`, entonces el modelo se ajustará a un modelo adicional al final del entrenamiento para asignar las puntuaciones de la función de decisión SVM a las probabilidades estimadas. . Básicamente, esto requiere utilizar una validación cruzada quíntuple para generar predicciones fuera de la muestra para cada instancia en el conjunto de entrenamiento y luego entrenar un modelo de `LogisticRegression`, por lo que ralentizará considerablemente el entrenamiento. Después de eso, los métodos `predict_proba()` y `predict_log_proba()` estarán disponibles.

## Clasificación SVM No-Lineal

Aunque los clasificadores SVM lineales son eficientes y a menudo funcionan sorprendentemente bien, muchos conjuntos de datos ni siquiera están cerca de ser separables linealmente. 

Un enfoque para manejar conjuntos de datos no lineales es agregar más características, como características polinómicas (como hicimos en el capítulo 4); en algunos casos esto puede resultar en un conjunto de datos separable linealmente. 

Considere el gráfico de la izquierda en la Figura 5-5: representa un conjunto de datos simple con una sola característica, x1. 
Este conjunto de datos no se puede separar linealmente, como se puede ver. Pero si añades una segunda característica x2 = (x1)2, el conjunto de datos 2D resultante es perfectamente separable linealmente.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0505.png)

(_Figura 5-5. Añadir características para hacer que un conjunto de datos sea separable linealmente_)

Para implementar esta idea usando Scikit-Learn, puede crear una canalización que contenga un transformador `PolynomialFeatures` (que se analiza en “Regresión polinomial”), seguido de un `StandardScaler` y un clasificador `LinearSVC`. 

Probemos esto en el conjunto de datos de las lunas, un conjunto de datos de juguete para clasificación binaria en el que los puntos de datos tienen la forma de dos lunas crecientes entrelazadas (consulte la Figura 5-6). Puedes generar este conjunto de datos usando la función `make_moons()`:

In [4]:
from sklearn.datasets import make_moons
from sklearn.preprocessing import PolynomialFeatures

X, y = make_moons(n_samples=100, noise=0.15, random_state=42)

polynomial_svm_clf = make_pipeline(
    PolynomialFeatures(degree=3),
    StandardScaler(),
    LinearSVC(C=10, max_iter=10_000, random_state=42)
)
polynomial_svm_clf.fit(X, y)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=3)),
                ('standardscaler', StandardScaler()),
                ('linearsvc',
                 LinearSVC(C=10, max_iter=10000, random_state=42))])

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0506.png)

(_Figura 5-6. Clasificador SVM lineal usando características polinómicas_)

## Núcleo polinómico (Polynomical Kernel)

La adición de características polinómicas es fácil de implementar y puede funcionar muy bien con todo tipo de algoritmos de aprendizaje automático (no solo con SVM). Dicho esto, en un grado polinómico bajo, este método no puede lidiar con conjuntos de datos muy complejos, y con un grado polinómico alto crea una gran cantidad de características, lo que hace que el modelo sea demasiado lento.

Afortunadamente, cuando se usan SVM se puede aplicar una técnica matemática casi milagrosa llamada el truco del núcleo (que se explica más adelante en este capítulo). El truco del núcleo hace posible obtener el mismo resultado que si hubieras añadido muchas características polinómicas, incluso con un grado muy alto, sin tener que añadirlas. Esto significa que no hay una explosión combinatoria en el número de características. Este truco es implementado por la clase SVC. Vamos a probarlo en el conjunto de datos de las lunas:

In [5]:
from sklearn.svm import SVC

poly_kernel_svm_clf = make_pipeline(StandardScaler(),
                                    SVC(kernel="poly", degree=3, coef0=1, C=5))
poly_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=5, coef0=1, kernel='poly'))])

Este código entrena un clasificador SVM utilizando un núcleo polinomial de tercer grado, representado a la izquierda en la Figura 5-7. A la derecha hay otro clasificador SVM que utiliza un núcleo polinomial de décimo grado. Obviamente, si su modelo está sobreajustado, es posible que desee reducir el grado del polinomio. Por el contrario, si no es suficiente, puedes intentar aumentarlo. El hiperparámetro `coef0` controla en qué medida el modelo está influenciado por términos de alto grado versus términos de bajo grado.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0507.png)

(_Figura 5-7. Clasificadores SVM con un núcleo polinómico_)

#### ------------------------------------ TIP -----------------------------------------
Aunque los hiperparámetros generalmente se ajustarán automáticamente (por ejemplo, usando la búsqueda aleatoria), es bueno tener una idea de lo que realmente hace cada hiperparámetro y cómo puede interactuar con otros hiperparámetros: de esta manera, puede reducir la búsqueda a un espacio mucho más pequeño.
#### -----------------------------------------------------------------------------------


## Características de simulitud
</br></br>
Otra técnica para abordar problemas no lineales es agregar características calculadas utilizando una función de similitud, que mide hasta qué punto cada instancia se parece a un punto de referencia en particular, como hicimos en el Capítulo 2 cuando agregamos las características de similitud geográfica. Por ejemplo, tomemos el conjunto de datos 1D de antes y agreguemos dos puntos de referencia en x1 = -2 y x1 = 1 (ver el gráfico de la izquierda en la Figura 5-8). A continuación, definiremos la función de similitud como el RBF gaussiano con γ = 0,3. Esta es una función en forma de campana que varía de 0 (muy lejos del punto de referencia) a 1 (en el punto de referencia).

Ahora estamos listos para calcular las nuevas características. Por ejemplo, veamos la instancia x1 = -1: se encuentra a una distancia de 1 del primer punto de referencia y 2 del segundo punto de referencia. Por lo tanto, sus nuevas características son x2 = exp(–0.3 × 12) ≈ 0.74 yx3 = exp(–0.3 × 22) ≈ 0.30. El gráfico de la derecha en la Figura 5-8 muestra el conjunto de datos transformado (dejando caer las características originales). Como puedes ver, ahora es linealmente separable.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0508.png)

(_Figuras 5-8. Características de similitud usando el RBF gaussiano_)

Puede que te preguntes cómo seleccionar los puntos de referencia. El enfoque más simple es crear un punto de referencia en la ubicación de todas y cada una de las instancias del conjunto de datos. Hacer eso crea muchas dimensiones y, por lo tanto, aumenta las posibilidades de que el conjunto de entrenamiento transformado sea separable linealmente. La desventaja es que un conjunto de entrenamiento con m instancias y n características se transforma en un conjunto de entrenamiento con m instancias y m características (suponiendo que se dejen caer las características originales). Si tu conjunto de entrenamiento es muy grande, terminas con un número igualmente grande de características.


## Núcleo RBF Gaussiano (RBF Gaussian kernel)

</br></br>
Al igual que el método de características polinómicas, el método de características de similitud puede ser útil con cualquier algoritmo de aprendizaje automático, pero puede resultar costoso desde el punto de vista computacional calcular todas las características adicionales (especialmente en conjuntos de entrenamiento grandes). Una vez más el truco del kernel hace su magia SVM, permitiendo obtener un resultado similar como si hubieras añadido muchas características de similitud, pero sin hacerlo realmente. Probemos la clase SVC con el núcleo Gaussiano RBF:

In [6]:
rbf_kernel_svm_clf = make_pipeline(StandardScaler(),
                                   SVC(kernel="rbf", gamma=5, C=0.001))
rbf_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=0.001, gamma=5))])

Este modelo está representado en la parte inferior izquierda de la Figura 5-9. Los otros gráficos muestran modelos entrenados con diferentes valores de los hiperparámetros gamma (γ) y C. El aumento de gamma hace que la curva en forma de campana sea más estrecha (consulte los gráficos de la izquierda en la Figura 5-8). Como resultado, el rango de influencia de cada instancia es menor: el límite de decisión termina siendo más irregular, moviéndose alrededor de instancias individuales. 

Por el contrario, un valor gamma pequeño hace que la curva en forma de campana sea más ancha: las instancias tienen un rango de influencia más amplio y el límite de decisión termina siendo más suave. Entonces γ actúa como un hiperparámetro de regularización: si su modelo está sobreajustado, debe reducir γ; si no es adecuado, debe aumentar γ (similar al hiperparámetro C).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0509.png)

(_Figura 5-9. Clasificadores SVM usando un núcleo RBF_)

Existen otros núcleos, pero se usan mucho más raramente. Algunos núcleos están especializados para estructuras de datos específicas. Los núcleos de cadena se utilizan a veces al clasificar documentos de texto o secuencias de ADN (por ejemplo, utilizando el núcleo de subsecuencia de cadena o núcleos basados en la distancia de Levenshtein).

#### ----------------------------------- TIP ------------------------------------------
Con tantos núcleos para elegir, ¿cómo puedes decidir cuál usar? Como regla general, siempre debes probar primero el kernel lineal. La clase `LinearSVC` es mucho más rápida que `SVC(kernel="linear")`, especialmente si el conjunto de entrenamiento es muy grande. Si no es demasiado grande, también debería probar SVM kernelizadas, comenzando con el kernel Gaussiano RBF; muchas veces funciona muy bien. Luego, si tiene tiempo libre y capacidad informática, puede experimentar con algunos otros núcleos utilizando la búsqueda de hiperparámetros. Si existen núcleos especializados para la estructura de datos de su conjunto de entrenamiento, asegúrese de probarlos también.
#### -----------------------------------------------------------------------------------

## Clases SVM y complejidad computacional

La clase `LinearSVC` se basa en la biblioteca `liblinear`, que implementa un algoritmo optimizado para SVM lineales.⁠1 No admite el truco del kernel, pero escala casi linealmente con la cantidad de instancias de entrenamiento y la cantidad de funciones. La complejidad del tiempo de entrenamiento es aproximadamente O (m × n). El algoritmo tarda más si se requiere una precisión muy alta. Esto está controlado por el hiperparámetro de tolerancia ϵ (llamado `tol` en Scikit-Learn). En la mayoría de las tareas de clasificación, la tolerancia predeterminada está bien.

La clase `SVC` se basa en la biblioteca `libsvm`, que implementa un algoritmo que admite el **truco del kernel**.⁠ 
La complejidad del tiempo de entrenamiento suele estar entre O(m2 × n) y O(m3 × n). 
Desafortunadamente, esto significa que se vuelve terriblemente lento cuando el número de instancias de entrenamiento aumenta (por ejemplo, cientos de miles de instancias), por lo que este algoritmo es mejor para conjuntos de entrenamiento no lineales de tamaño pequeño o mediano. Se adapta bien a la cantidad de funciones, especialmente con funciones escasas (es decir, cuando cada instancia tiene pocas funciones distintas de cero). En este caso, el algoritmo escala aproximadamente con el número promedio de características distintas de cero por instancia.

La clase `SGDClassifier` también realiza una clasificación de margen grande de forma predeterminada, y sus hiperparámetros, especialmente los hiperparámetros de regularización (alfa y `penality`) y `learning_rate`, se pueden ajustar para producir resultados similares a los de las SVM lineales. Para el entrenamiento, utiliza un descenso de gradiente estocástico (consulte el Capítulo 4), que permite el aprendizaje incremental y utiliza poca memoria, por lo que puede usarlo para entrenar un modelo en un conjunto de datos grande que no cabe en la RAM (es decir, para modelos fuera del núcleo). aprendiendo). Además, se escala muy bien, ya que su complejidad computacional es O (m × n). La Tabla 5-1 compara las clases de clasificación SVM de Scikit-Learn.

Tabla 5-1. Comparación de las clases de Scikit-Learn para la clasificación SVM:

| Clase         | Complejidad temporal      | Soporte fuera del kernel | Requiere escalado | Truco del kernel |
|---------------|---------------------------|--------------------------|-------------------|------------------|
| LinearSVC     | O(m × n)                  | No                       | Si                | No               |
| SVC           | de O(m² × n)  a O(m³ × n) | No                       | Si                | Si               |
| SGDClassifier | O(m × n)                  | Si                       | Si                | No               |

## Regresión SVM

Para usar SVM para la regresión en lugar de la clasificación, el truco es ajustar el objetivo: en lugar de tratar de encajar la calle más grande posible entre dos clases mientras se limitan las violaciones de margen, la regresión SVM intenta encajar tantas instancias como sea posible en la calle mientras se limitan las violaciones de margen (es decir, instancias fuera de la calle). El ancho de la calle está controlado por un hiperparámetro, ε. 

La figura 5-10 muestra dos modelos de regresión SVM lineal entrenados en algunos datos lineales, uno con un margen pequeño (ε = 0,5) y el otro con un margen más grande (ε = 1,2).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0510.png)

(_Figura 5-10. Regresión SVM_)


La reducción de ε aumenta el número de vectores de soporte, lo que regulariza el modelo. Además, si agrega más instancias de entrenamiento dentro del margen, no afectará a las predicciones del modelo; por lo tanto, se dice que el modelo es ε-insensible.

Puede utilizar la clase LinearSVR de Scikit-Learn para realizar una regresión SVM lineal. El siguiente código produce el modelo representado a la izquierda en la Figura 5-10:

In [7]:
from sklearn.svm import LinearSVR

X, y = [...]  # a linear dataset
svm_reg = make_pipeline(StandardScaler(),
                        LinearSVR(epsilon=0.5, random_state=42))
svm_reg.fit(X, y)

ValueError: not enough values to unpack (expected 2, got 1)

Para abordar las tareas de regresión no lineal, puede utilizar un modelo SVM kernelizado. La figura 5-11 muestra la regresión SVM en un conjunto de entrenamiento cuadrático aleatorio, utilizando un núcleo polinómico de segundo grado. Hay cierta regularización en el gráfico izquierdo (es decir, un pequeño valor `C`), y mucho menos en el gráfico derecho (es decir, un gran valor `C`). 

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0511.png)

(_Figura 5-11. Regresión SVM utilizando un núcleo polinómico de segundo grado_)


El siguiente código utiliza la clase SVR de Scikit-Learn (que admite el truco del kernel) para producir el modelo representado a la izquierda en la Figura 5-11:

In [8]:
from sklearn.svm import SVR

X, y = [...]  # a quadratic dataset
svm_poly_reg = make_pipeline(StandardScaler(),
                             SVR(kernel="poly", degree=2, C=0.01, epsilon=0.1))
svm_poly_reg.fit(X, y)

ValueError: not enough values to unpack (expected 2, got 1)

La clase SVR es el equivalente de regresión de la clase SVC y la clase LinearSVR es el equivalente de regresión de la clase `LinearSVC`. La clase `LinearSVR` escala linealmente con el tamaño del conjunto de entrenamiento (al igual que la clase LinearSVC), mientras que la clase SVR se vuelve demasiado lenta cuando el conjunto de entrenamiento crece mucho (al igual que la clase SVC).

#### ----------------------- NOTA ----------------------------------
Los SVM también se pueden utilizar para la detección de novedades, como verá en el capítulo 9.
#### -----------------------------------------------------------------

El resto de este capítulo explica cómo los SVM hacen predicciones y cómo funcionan sus algoritmos de entrenamiento, comenzando con los clasificadores lineales de SVM. Si acabas de empezar con el aprendizaje automático, puedes saltarte esto de forma segura e ir directamente a los ejercicios al final de este capítulo, y volver más tarde cuando quieras obtener una comprensión más profunda de los SVM.


## Bajo el capó de los clasificadores SVM lineales

</br></br>
Un clasificador SVM lineal predice la clase de una nueva instancia x calculando primero la función de decisión θ⊺ x = θ0 x0 + ⋯ + θn xn, donde x0 es la característica de sesgo (siempre igual a 1). Si el resultado es positivo, entonces la clase predicha ŷ es la clase positiva (1); de lo contrario es la clase negativa (0). Esto es exactamente como la Regresión Logística (que se analiza en el Capítulo 4).

#### ------------------------------- NOTA ----------------------------------
Hasta ahora, he utilizado la convención de poner todos los parámetros del modelo en un vector θ, incluido el término de sesgo θ0 y los pesos de la característica de entrada θ1 a θn. Esto requirió agregar una entrada de sesgo x0 = 1 a todas las instancias. Otra convención muy común es separar el término sesgo b (igual a θ0) y el vector de pesos de características w (que contiene θ1 a θn). En este caso, no es necesario agregar ninguna característica de sesgo a los vectores de la característica de entrada, y la función de decisión del SVM lineal es igual a w⊺ x + b = w1 x1 + ⋯ + wn xn + b. Usaré esta convención durante el resto de este libro.

#### -------------------------------------------------------------------------

Por lo tanto, hacer predicciones con un clasificador SVM lineal es bastante sencillo. ¿Qué tal el entrenamiento? Esto requiere encontrar el vector de pesos w y el término de sesgo b que hacen que la calle, o margen, sea lo más ancha posible, al tiempo que se limita el número de violaciones de margen. Comencemos con el ancho de la calle: para hacerla más grande, tenemos que hacerla más pequeña. Esto puede ser más fácil de visualizar en 2D, como se muestra en la Figura 5-12. Definamos los límites de la calle como los puntos en los que la función de decisión es igual a -1 o +1. En el gráfico izquierdo, el peso w1 es 1, por lo que los puntos en los que w1 x1 = -1 o +1 son x1 = -1 y +1: por lo tanto, el tamaño del margen es 2. En el gráfico de la derecha, el peso es 0,5, por lo que los puntos en los que w1 x1 = -1 o +1 son x1 = -2 y +2: el tamaño del margen es 4. Por lo tanto, tenemos que mantenernos lo más pequeños posible. Tenga en cuenta que el término sesgo b no influye en el tamaño del margen: ajustarlo solo desplaza el margen, sin afectar su tamaño.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0512.png)

(_Figura 5-12. Un vector de peso más pequeño da como resultado un margen mayor_)

También queremos evitar las violaciones del margen, por lo que necesitamos que la función de decisión sea mayor que 1 para todas las instancias de entrenamiento positivas y inferior a -1 para las instancias de entrenamiento negativo. Si definimos t(i) = -1 para instancias negativas (cuando y(i) = 0) y t(i) = 1 para instancias positivas (cuando y(i) = 1), entonces podemos escribir esta restricción como t(i)(w⊺ x(i) + b) ≥ 1 para todas las instancias.

Por lo tanto, podemos expresar el objetivo del clasificador SVM lineal de margen duro como el problema de optimización restringida en la Ecuación 5-1.

### Ecuación 5-1. Objetivo del clasificador SVM lineal de margen duro

<a href="https://imgbb.com/"><img src="https://i.ibb.co/P6XfYYw/Captura-de-pantalla-2023-09-09-a-las-6-03-38.png" alt="Captura-de-pantalla-2023-09-09-a-las-6-03-38" border="0"></a>

#### ------------------------------- NOTA --------------------------------------------
Estamos minimizando ½ w⊺ w, que es igual a ½∥ w ∥2, en lugar de minimizar ∥ w ∥ (la norma de w). De hecho, ½∥ w ∥2 tiene una derivada agradable y simple (es solo w), mientras que ∥ w ∥ no es diferenciable en w = 0. Los algoritmos de optimización a menudo funcionan mucho mejor en funciones diferenciables.
#### -----------------------------------------------------------------------------------

Para obtener el objetivo de margen blando, necesitamos introducir una variable de holgura ζ(i) ≥ 0 para cada instancia:⁠3 ζ(i) mide cuánto se permite que la i-i instancia viole el margen. Ahora tenemos dos objetivos contradictorios: hacer que las variables de holgura sean lo más pequeñas posible para reducir las violaciones del margen, y hacer que ½ w⊺ w sea lo más pequeña posible para aumentar el margen. Aquí es donde entra en juego el hiperparámetro C: nos permite definir la compensación entre estos dos objetivos. Esto nos da el problema de optimización restringida en la ecuación 5-2.

### Ecuación 5-2. Objetivo del clasificador SVM lineal de margen suave

<a href="https://ibb.co/WVJ3WB6"><img src="https://i.ibb.co/y4jRngX/Captura-de-pantalla-2023-09-09-a-las-6-07-22.png" alt="Captura-de-pantalla-2023-09-09-a-las-6-07-22" border="0"></a>

Los problemas de margen duro y margen blando son ambos problemas de optimización cuadrática convexa con restricciones lineales. Tales problemas se conocen como problemas de programación cuadrática (QP). Muchos solucionadores listos para usar están disponibles para resolver problemas de QP utilizando una variedad de técnicas que están fuera del alcance de este libro.

Usar un solucionador de QP es una forma de entrenar una SVM. Otra es utilizar el descenso de gradiente para minimizar la pérdida de bisagra o la pérdida de bisagra al cuadrado (consulte la Figura 5-13). Dada una instancia x de la clase positiva (es decir, con t = 1), la pérdida es 0 si la salida s de la función de decisión (s = w⊺ x + b) es mayor o igual a 1. Esto sucede cuando la El ejemplo está fuera de la calle y en el lado positivo. Dada una instancia de la clase negativa (es decir, con t = –1), la pérdida es 0 si s ≤ –1. Esto sucede cuando la instancia está fuera de la calle y en el lado negativo. Cuanto más lejos esté una instancia del lado correcto del margen, mayor será la pérdida: crece linealmente para la pérdida de bisagra y cuadráticamente para la pérdida de bisagra al cuadrado. Esto hace que la pérdida de bisagra al cuadrado sea más sensible a los valores atípicos. Sin embargo, si el conjunto de datos está limpio, tiende a converger más rápido. De forma predeterminada, LinearSVC usa la pérdida de bisagra al cuadrado, mientras que SGDClassifier usa la pérdida de bisagra. Ambas clases le permiten elegir la pérdida configurando el hiperparámetro de pérdida en "hinge" o "squared_hinge". El algoritmo de optimización de la clase SVC encuentra una solución similar a la de minimizar la pérdida de bisagra.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_0513.png)

(_Figura 5-13. La pérdida de la bisagra (izquierda) y la pérdida de la bisagra cuadrada (derecha)_)

A continuación, miramos otra forma de entrenar a un clasificador SVM lineal: resolver el problema dual.

## El problema dual

Dado un problema de optimización restringido, conocido como el problema primordial, es posible expresar un problema diferente pero estrechamente relacionado, llamado su problema dual. La solución al problema dual normalmente da un límite inferior a la solución del problema primario, pero bajo algunas condiciones puede tener la misma solución que el problema primario. 

Afortunadamente, el problema de SVM cumple con estas condiciones,⁠ por lo que puede elegir resolver el problema primario o el problema dual; ambos tendrán la misma solución. La ecuación 5-3 muestra la forma dual del objetivo SVM lineal. Si está interesado en saber cómo derivar el problema dual del problema primordial, consulte la sección de material adicional en el cuaderno de este capítulo.

### Ecuación 5-3. Forma dual del objetivo SVM lineal

<a href="https://ibb.co/m6jBWTP"><img src="https://i.ibb.co/kQ7yCBP/Captura-de-pantalla-2023-09-10-a-las-5-10-32.png" alt="Captura-de-pantalla-2023-09-10-a-las-5-10-32" border="0"></a>

Una vez que encuentres el vector α^ que minimiza esta ecuación (usando un solucionador de QP), use la ecuación 5-4 para calcular el **W^** Y **b^** que minimizan el problema primordial. En esta ecuación, _ns_ representa el número de vectores de soporte.

### Ecuación 5-4. De la solución dual a la solución primaria

<a href="https://imgbb.com/"><img src="https://i.ibb.co/mRZDnp4/Captura-de-pantalla-2023-09-10-a-las-5-15-31.png" alt="Captura-de-pantalla-2023-09-10-a-las-5-15-31" border="0"></a>

El problema dual es más rápido de resolver que el primario cuando el número de instancias de entrenamiento es menor que el número de características. Lo que es más importante, el problema dual hace posible el truco del núcleo, mientras que el problema primario no lo hace. Entonces, ¿qué es este truco del núcleo, de todos modos?


## SVM con Kernel

</br></br>
Supongamos que desea aplicar una transformación polinómica de segundo grado a un conjunto de entrenamiento bidimensional (como el conjunto de entrenamiento de lunas), y luego entrenar un clasificador SVM lineal en el conjunto de entrenamiento transformado. La ecuación 5-5 muestra la función de mapeo polinomial de segundo grado φ que desea aplicar.


### Ecuación 5-5. Mapeo polinómico de segundo grado

<a href="https://imgbb.com/"><img src="https://i.ibb.co/rF0nd5b/Captura-de-pantalla-2023-09-10-a-las-5-18-26.png" alt="Captura-de-pantalla-2023-09-10-a-las-5-18-26" border="0"></a>

Ten en cuenta que el vector transformado es 3D en lugar de 2D. Ahora veamos qué sucede con un par de vectores 2D, **a** y **b**, si aplicamos este mapeo polinómico de segundo grado y luego calculamos el producto de puntos⁠ de los vectores transformados (ver Ecuación 5-6).

### Ecuación 5-6. Truco del kernel para un mapeo polinómico de segundo grado

<a href="https://imgbb.com/"><img src="https://i.ibb.co/d48JxFm/Captura-de-pantalla-2023-09-10-a-las-5-20-22.png" alt="Captura-de-pantalla-2023-09-10-a-las-5-20-22" border="0"></a>

¿Qué tal eso? El producto de puntos de los vectores transformados es igual al cuadrado del producto de puntos de los vectores originales: φ(a)⊺ φ(b) = (a⊺ b)2.

Aquí está la idea clave: si aplica la transformación φ a todas las instancias de entrenamiento, entonces el problema dual (ver Ecuación 5-3) contendrá el producto de puntos φ(x(i))⊺φ(x(j)). Pero si φ es la transformación polinómica de segundo grado definida en la ecuación 5-5, entonces puedes reemplazar este producto de puntos de los vectores transformados simplemente por **(Xˆ(i)ˆT * Xˆ(j)ˆT)ˆ2**. 

Por lo tanto, no necesita transformar las instancias de entrenamiento en absoluto; simplemente reemplace el producto de punto por su cuadrado en la ecuación 5-3. 
El resultado será estrictamente el mismo que si hubieras pasado por la molestia de transformar el conjunto de entrenamiento y luego ajustar un algoritmo SVM lineal, pero este truco hace que todo el proceso sea mucho más eficiente desde el punto de vista computacional.

La función K(a, b) = (a⊺ b)2 es un núcleo polinómico de segundo grado. En el aprendizaje automático, un núcleo es una función capaz de calcular el producto de puntos φ(a)⊺ φ(b), basado solo en los vectores originales a y b, sin tener que calcular (o incluso conocer) la transformación φ. La ecuación 5-7 enumera algunos de los núcleos más utilizados.

### Ecuación 5-7. Kérnels comunes

<a href="https://imgbb.com/"><img src="https://i.ibb.co/DKy3YSb/Captura-de-pantalla-2023-09-10-a-las-5-24-05.png" alt="Captura-de-pantalla-2023-09-10-a-las-5-24-05" border="0"></a>

#### -------------------------- TEOREMA DE MERCER ---------------------------
De acuerdo con el teorema de Mercer, si una función K(a, b) respeta algunas condiciones matemáticas llamadas condiciones de Mercer (por ejemplo, K debe ser continua y simétrica en sus argumentos para que K(a, b) = K(b, a), etc.), entonces existe una función φ que asigna a y b a otro espacio (posiblemente con dimensiones mucho más altas) de tal manera que K(a, b) = φ(a)⊺ φ(b). Puedes usar K como núcleo porque sabes que φ existe, incluso si no sabes qué es φ. En el caso del núcleo RBF gaussiano, se puede demostrar que φ asigna cada instancia de entrenamiento a un espacio de dimensión infinita, ¡por lo que es bueno que no necesites realizar el mapeo!

Tenga en cuenta que algunos núcleos de uso frecuente (como el núcleo sigmoide) no respetan todas las condiciones de Mercer, pero generalmente funcionan bien en la práctica.
#### ----------------------------------------------------------------------------------

Todavía hay un extremo suelto que debemos atar. La ecuación 5-4 muestra cómo pasar de la solución dual a la solución primaria en el caso de un clasificador SVM lineal. Pero si aplicas el truco del núcleo, terminas con ecuaciones que incluyen φ(x(i)). De hecho,
W^ debe tener el mismo número de dimensiones que φ(x(i)), que puede ser enorme o incluso infinita, por lo que no se puede calcular. Pero, ¿cómo puedes hacer predicciones sin saberlo? W^ ? 

Bueno, la buena noticia es que puedes conectar la fórmula para
W^ de la ecuación 5-4 en la función de decisión para una nueva instancia x(n), y se obtiene una ecuación con solo productos de puntos entre vectores de entrada. Esto hace posible usar el truco del núcleo (Ecuación 5-8).

### Ecuación 5-8. Hacer predicciones con un SVM estandarizado
</br></br>
<a href="https://imgbb.com/"><img src="https://i.ibb.co/rdx9h3j/Captura-de-pantalla-2023-09-10-a-las-5-28-34.png" alt="Captura-de-pantalla-2023-09-10-a-las-5-28-34" border="0"></a>

Tenga en cuenta que, dado que α(i) ≠ 0 solo para vectores de soporte, hacer predicciones implica calcular el producto de puntos del nuevo vector de entrada x(n) con solo los vectores de soporte, no con todas las instancias de entrenamiento. Por supuesto, necesitas usar el mismo truco para calcular el término sesgo
b^ (Ecuación 5-9).


### Ecuación 5-9. Usando el truco del kernel para calcular el término sesgo
</br></br>
<a href="https://ibb.co/sRN3PTj"><img src="https://i.ibb.co/LRcpPTS/Captura-de-pantalla-2023-09-10-a-las-5-30-10.png" alt="Captura-de-pantalla-2023-09-10-a-las-5-30-10" border="0"></a>

Si estás empezando a tener dolor de cabeza, eso es perfectamente normal: es un desafortunado efecto secundario del truco del núcleo.

#### ---------------------------------- NOTA -----------------------------------------
También es posible implementar SVM con núcleo en línea, capaces de aprendizaje incremental, como se describe en los documentos "Aprendizaje automático vectorial de soporte incremental y decrecional"⁠7 y "Clasificadores de núcleo rápido con aprendizaje en línea y activo".⁠8 Estos SVM con núcleo se implementan en Matlab y C++. 
Pero para problemas no lineales a gran escala, es posible que desee considerar el uso de bosques aleatorios (véase el capítulo 7) o redes neuronales (véase la parte II).
#### -----------------------------------------------------------------------------------

# Ejercicios
</br>
1. ¿Cuál es la idea fundamental detrás de las máquinas vectoriales de soporte?

2. ¿Qué es un vector de apoyo?

3. ¿Por qué es importante escalar las entradas cuando se utilizan SVM?

4. ¿Puede un clasificador SVM generar una puntuación de confianza cuando clasifica una instancia? ¿Qué hay de una probabilidad?

5. Como debes elegir entre LinearSVC, SVC, and SGDClassifier?

6. Supongamos que has entrenado a un clasificador SVM con un núcleo RBF, pero parece que no se ajusta al conjunto de entrenamiento. ¿Deberías aumentar o disminuir γ (gamma)? ¿Qué pasa con C?

7. ¿Qué significa que un modelo sea ε-insensible?

8. ¿Qué sentido tiene usar el truco del núcleo?

9. Train a LinearSVC on a linearly separable dataset. Then train an SVC and a SGDClassifier on the same dataset. See if you can get them to produce roughly the same model.

10. Entrena un clasificador SVM en el conjunto de datos de vinos, que puedes cargar usandosklearnsklearn.datasets.load_wine(). Este conjunto de datos contiene los análisis químicos de 178 muestras de vino producidas por 3 cultivadores diferentes: el objetivo es entrenar un modelo de clasificación capaz de predecir el cultivador basado en el análisis químico del vino. Dado que los clasificadores SVM son clasificadores binarios, tendrá que usar uno por todos para clasificar las tres clases. ¿Qué precisión puedes alcanzar?

11. Entrena y ajusta un regresor SVM en el conjunto de datos de viviendas de California. Puedes usar el conjunto de datos original en lugar de la versión ajustada que usamos en el capítulo 2, que puedes cargar usandosklearnsklearn.datasets.fetch_california_housing(). Los objetivos representan cientos de miles de dólares. Dado que hay más de 20 000 instancias, los SVM pueden ser lentos, por lo que para el ajuste de hiperparámetros debe usar muchas menos instancias (por ejemplo, 2.000) para probar muchas más combinaciones de hiperparámetros. ¿Cuál es el RMSE de tu mejor modelo?

Las soluciones a estos ejercicios están disponibles al final del cuaderno de este capítulo, en https://homl.info/colab3.